In [2]:
from box_wrapper import SigmoidBoxTensor, BoxTensor, TBoxTensor, DeltaBoxTensor, MinDeltaBoxesOnTorus
from modules import BoxEmbedding
from utils import log1mexp
from allennlp.modules.token_embedders import Embedding
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim

# We will consider 3 entities in this toy example.
- Positive edge: (0, 1) (Just repeated this one twice in the dataset)
- Negative edges: (0, 2), (0,2), (1,2), (2,1)
- for positive edges - P(A|B) = 1 (These can be any valid probability values)
- for negative edges - P(B|A) = 0 (These can be any valid probability values)

In [ ]:
num_entities = 3

In [ ]:
def get_data():
    head = torch.LongTensor([0, 0])
    tail = torch.LongTensor([1, 1])
    neg_head = torch.LongTensor([0, 2, 1, 2])
    neg_tail = torch.LongTensor([2, 1, 2, 0])
    return head, tail, neg_head, neg_tail

### The training loop 

In [ ]:
def train_loop(my_net, max_epoch=10000, log_freq=2000):
    embeddings = []
    gradients = []
    for epoch in range(max_epoch):
        #for data in trainset:
        my_net.zero_grad()
        head, tail, neg_head, neg_tail = get_data()
        output = my_net(head, tail, neg_head, neg_tail)
        if epoch%log_freq == 0:
            print(output)
        output.backward()
        xc = [ele.grad for ele in my_net.parameters()]
        gradients.append(xc)
        optimizer.step()
    return embeddings , gradients

### Box embedding model that implements the Algorithm 1 described in the paper.

In [ ]:
class toy_smooth_box(nn.Module):
    def __init__(self, softbox_temp, gumbel_beta, regularisation=0.0):
        super().__init__()
        self.box_type = 'BoxTensor'
        self.box = BoxEmbedding(
            num_embeddings=num_entities,
            box_embedding_dim=2,
            box_type=self.box_type,
            sparse=False,
            init_interval_center = 0.2,
            init_interval_delta = 0.01)
        self.embedding_dim = 2
        self.softbox_temp = softbox_temp
        self.gumbel_beta = gumbel_beta
        self.loss_f = torch.nn.NLLLoss(reduction='mean')
        self.label = torch.tensor([1, 1, 0, 0, 0, 0], dtype=torch.long)
        self.regularization_weight = regularisation

    def forward(self, head, tail, neg_head, neg_tail):
        pos_head = self.box(head)
        pos_tail = self.box(tail)
        neg_head = self.box(neg_head)
        neg_tail = self.box(neg_tail)
        pos_score = self.get_scores(pos_head, pos_tail)
        neg_score = self.get_scores(neg_head, neg_tail)
        loss = self.get_loss(torch.cat([pos_score, neg_score], 0), self.label)
        return loss
        
    def _get_score(self, head, tail):
        head_tail_box_vol = head.intersection_log_soft_volume(
            tail, temp=self.softbox_temp)
        # score = tail_head_relation_box_vol - tail_relation_box.log_soft_volume(
        #    temp=self.softbox_temp)
        score = head_tail_box_vol - tail.log_soft_volume(temp=self.softbox_temp)

        return score
    
    def get_loss(self, scores, label):
        log_p = scores
        log1mp = log1mexp(log_p)
        logits = torch.stack([log1mp, log_p], dim=-1)
        logits = logits.view(6, 2)
        loss = self.loss_f(logits, label)
        if torch.isnan(loss).any():
            breakpoint()
        return loss
    
    def get_scores(self, head, tail) -> torch.Tensor:
        p = self._get_score(head, tail)
        return p

### Gumbel Box Model

In [ ]:
class toy_box_gumbel(toy_smooth_box):
    def _get_score(self, head: BoxTensor, tail: BoxTensor) -> torch.Tensor:
        intersection_box = head.gumbel_intersection(tail, gumbel_beta=self.gumbel_beta)
        intersection_volume = intersection_box._log_soft_volume_adjusted(
            intersection_box.z, intersection_box.Z , temp= self.softbox_temp)

        tail_volume= tail._log_soft_volume_adjusted(tail.z, tail.Z, temp= self.softbox_temp, gumbel_beta = self.gumbel_beta)
        
        score = (intersection_volume - tail_volume)
        if len(np.where(score>0)[0]):
            breakpoint()
        return score

In [ ]:
class toy_box_gumbel_with_bessel_volume(toy_smooth_box):
    def _get_score(self, head: BoxTensor, tail: BoxTensor) -> torch.Tensor:
        intersection_box = head.gumbel_intersection(tail, gumbel_beta=self.gumbel_beta)
        intersection_vol = intersection_box._log_bessel_volume(intersection_box.z,
            intersection_box.Z, gumbel_beta=self.gumbel_beta)
        tail_vol = tail._log_bessel_volume(tail.z, tail.Z, gumbel_beta=self.gumbel_beta)
        score = intersection_vol - tail_vol
        if len(np.where(score>0)[0]):
            breakpoint()
        return score

In [ ]:
my_net = toy_box_gumbel(1.0, 1.0)
optimizer = optim.Adam(my_net.parameters(), lr = 0.01)
embeddings, grads = train_loop(my_net, 1000, 30)

In [ ]:
my_net = toy_box_gumbel_with_bessel_volume(1.0, 1.0)
optimizer = optim.Adam(my_net.parameters(), lr = 0.01)
embeddings, grads = train_loop(my_net, 1000, 30)

In [ ]:
my_net = toy_smooth_box(1.0, 1.0)
optimizer = optim.Adam(my_net.parameters(), lr = 0.01)
embeddings, grads = train_loop(my_net, 1000, 30)

### Test volume for BoxTensor

In [3]:
test_box = BoxTensor(torch.tensor([[0.0,0.0], [1.0,1.0]]))

In [4]:
print(f"Min: {test_box.z}, Max: {test_box.Z}")

Min: tensor([0., 0.]), Max: tensor([1., 1.])


- Volume = (1 - 0) * (1 - 0) = 1.0
- log volume = log (1.0) = 0.0

In [5]:
print(f"hard volume: {test_box.log_clamp_volume()}")

hard volume: 0.0


- Soft volume tends to converge to its hard value as temp decreases 

In [14]:
print(f"Soft volume with temp 1/1.0: {test_box.log_soft_volume(temp=1.)}")
print(f"Soft volume with temp 1/10.0: {test_box.log_soft_volume(temp=10.)}")
print(f"Soft volume with temp 1/50.0: {test_box.log_soft_volume(temp=50.)}")

Soft volume with temp 1/1.0: 0.5450276732444763
Soft volume with temp 1/10.0: 9.059885087481234e-06
Soft volume with temp 1/50.0: 0.0


- Soft volume with gumbel distribution tends to converge to its hard value as gumbel beta decreases

In [15]:
print(f"Soft volume with temp 1/1.0: {test_box.log_soft_volume_adjusted(temp=50., gumbel_beta = 1.)}")
print(f"Soft volume with temp 1/10.0: {test_box.log_soft_volume_adjusted(temp=50., gumbel_beta = 0.01)}")
print(f"Soft volume with temp 1/50.0: {test_box.log_soft_volume_adjusted(temp=50., gumbel_beta = 0.0001)}")
print(f"Soft volume with temp 1/50.0: {test_box.log_soft_volume_adjusted(temp=50., gumbel_beta = 0.00001)}")
print(f"Soft volume with temp 1/50.0: {test_box.log_soft_volume_adjusted(temp=50., gumbel_beta = 0.0000001)}")
print(f"Soft volume with temp 1/50.0: {test_box.log_soft_volume_adjusted(temp=50., gumbel_beta = 0.0000000001)}")

Soft volume with temp 1/1.0: -23.267623901367188
Soft volume with temp 1/10.0: -0.02322288043797016
Soft volume with temp 1/50.0: -0.00023092172341421247
Soft volume with temp 1/50.0: -2.3126736778067425e-05
Soft volume with temp 1/50.0: -2.384185791015625e-07
Soft volume with temp 1/50.0: 0.0


### Adjustment in log_soft_volume_adjusted
- The gumbel boxes are initiazed by its position parameters(mu). However the mean is shift by Euler_Mascheroni_Constant , i.e., mean = mu + Euler_Mascheroni_Constant * gumbel_beta.
- As if the original box parameters would be shifted inwards by (Euler_Mascheroni_Constant * gumbel_beta) amount in both min and max.
- Thus we need to adjust that in our volume calculation.
- Verify that below.

- More on that constant - https://en.wikipedia.org/wiki/Euler%E2%80%93Mascheroni_constant
- More mean of gumbel dist - https://en.wikipedia.org/wiki/Gumbel_distribution (look at the property table)

In [33]:
Euler_Mascheroni_Constant = euler_gamma = 0.577215664901532
beta = 1.
# Calulate the adjusted volume of the given box
test_box = BoxTensor(torch.tensor([[0.0,0.0], [100.0,100.0]]))
adjusted_volume_of_box =torch.exp(test_box.log_soft_volume_adjusted(temp=50.0, gumbel_beta = beta))

# Calulated the volume of the adjusted box
test_box = BoxTensor(torch.tensor([[0.0 + euler_gamma * beta ,0.0 + euler_gamma * beta], 
                                   [100.0 - euler_gamma * beta, 100.0 - euler_gamma * beta]]))
volume_of_adjusted_box = torch.exp(test_box.log_soft_volume(temp=50.0))
assert volume_of_adjusted_box == adjusted_volume_of_box